In [1]:
#cells will fill entire width of the browser
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

#Tells Jupyter to reload custom classes from scratch everytime an import cell is run, if you edit a custom class
#between imports Jupyter would otherwise need to be restarted completely. Buyer beware: old class objects in the 
#current namespace will cause errors at execution
%load_ext autoreload
%autoreload 2

#switches matplotlib to show plots in the browser rather than opening a new window
%matplotlib inline

#always forget to do this for better looking plots
import seaborn
seaborn.set()

In [39]:
import matplotlib.pyplot as plt
import os
import numpy as np
import datetime
from statsmodels.tsa import stattools
from sklearn import preprocessing
from sklearn.linear_model import Ridge
import random
import copy
import scipy
import sklearn.metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

In [34]:
type(False) == np.ndarray

False

In [35]:
#Bayesian classifier
def mat_C(x, V=False):
    #x shape = px1
    if type(V) != np.ndarray:
        V = np.eye(x.shape[0])
    C = np.dot(x, x.T) + V
    return(C)

def mat_D(x, y, A, V=False):
    #x shape px1, y shape nx1, A nxp
    if type(V) != np.ndarray:
        V = np.eye(x.shape[0])
    D = np.dot(y, x.T) + A.dot(V)
    return(D)

def likelihood_point(x, y, A, V=False):
    C = mat_C(x, V)
    D = mat_D(x, y, A, V)
    C_inv = np.linalg.inv(C)
    const = x.shape[0]*np.log(np.linalg.det(C_inv))
    var = np.trace(y.dot(y.T) - 2*A.dot(x).dot(y.T) + x.T.dot(A.T).dot(A).dot(x))
    var2 = np.trace(y.dot(y.T) + A.dot(A.T) - D.dot(C_inv).dot(D.T))   #mixed term doesn't look right
    out = var - var2 + const
    return(out)

In [95]:
#initialize A, PD
A = np.array([[0.9, 0.0],
              [0.0, -0.4]])

V = np.array([[1.0, -0.001],
              [-0.001, 0.02]])

V_inv = np.linalg.inv(V)

lags = [10]
MC = 100#500
num_samps = 500

num_samps_list = np.arange(0,20,1)#[0,5,10,20,50,100]

In [96]:
np.linalg.eig(V)

(array([1.00000102, 0.01999898]), array([[ 0.99999948,  0.00102041],
        [-0.00102041,  0.99999948]]))

In [ ]:
mc_output_by_lag = {}

for M in lags:
    print("Lag round: ", M)
    mc_output_by_lag[M] = np.zeros((MC,3))
    for j in range(MC):
        if j % 10 == 0:
            print("Matrix initialization round: ", j)
        #B = A + randomly sample an element of A to perturb by N(0,1)
        #e = np.unravel_index(np.random.choice([i for i in range(A.shape[0]*A.shape[1])]), dims=A.shape)
        B = copy.copy(A) + np.vstack((np.random.multivariate_normal(np.array([0,0]),V), np.random.multivariate_normal(np.array([0,0]),V)))
        #B[e] += np.random.normal(0,1)
        mc_output_by_lag[M][j,0] = np.linalg.norm(B-A,'fro')
        
        #generate a bunch of samples
        X_s = np.random.normal(size=(2,num_samps))##, size=(2,1000))
        Y_s = A.dot(X_s) + np.asarray([ np.random.normal(0, np.eye(2))[:,0] for i in range(num_samps) ]).T
        Y_s_b = B.dot(X_s) + np.asarray([ np.random.normal(0, np.eye(2))[:,0] for i in range(num_samps) ]).T
        
        f_bin = []
        nf_bin = []
        
        for i in range(X_s.shape[1] - M):
            #for each sample
            nf_sum = [0.0]
            f_sum = [0.0]
            

            for k in range(M):
                #collect likelihood across lag
                nf = likelihood_point(np.expand_dims(X_s[:,i+k], axis=1), np.expand_dims(Y_s[:,i+k], axis=1), A)
                f = likelihood_point(np.expand_dims(X_s[:,i+k], axis=1), np.expand_dims(Y_s_b[:,i+k], axis=1), A)

                nf_sum.append(nf_sum[-1] + nf)
                f_sum.append(f_sum[-1] + f)


            if f_sum[-1] > 0:
                f_bin.append(1)
            else:
                f_bin.append(0)

            if nf_sum[-1] > 0:
                nf_bin.append(1)
            else:
                nf_bin.append(0)
                    
        pred = np.concatenate((f_bin, nf_bin))
        true = np.concatenate((np.ones(np.asarray(f_bin).shape), np.zeros(np.asarray(nf_bin).shape)))

        p = sklearn.metrics.precision_score(true, pred)
        r = sklearn.metrics.recall_score(true, pred)
                
        mc_output_by_lag[M][j,1] = p
        mc_output_by_lag[M][j,2] = r

Lag round:  10
Matrix initialization round:  0
Matrix initialization round:  10
Matrix initialization round:  20
Matrix initialization round:  30
Matrix initialization round:  40
Matrix initialization round:  50


In [ ]:
mc_output_by_lag_skew = {}

for M in lags:
    print("Lag round: ", M)
    mc_output_by_lag_skew[M] = np.zeros((MC,3))
    for j in range(MC):
        if j % 10 == 0:
            print("Matrix initialization round: ", j)
        #B = A + randomly sample an element of A to perturb by N(0,1)
        #e = np.unravel_index(np.random.choice([i for i in range(A.shape[0]*A.shape[1])]), dims=A.shape)
        B = copy.copy(A) + np.vstack((np.random.multivariate_normal(np.array([0,0]),V), np.random.multivariate_normal(np.array([0,0]),V)))
        #B[e] += np.random.normal(0,1)
        mc_output_by_lag_skew[M][j,0] = np.linalg.norm(B-A,'fro')
        
        #generate a bunch of samples
        X_s = np.random.normal(size=(2,num_samps))##, size=(2,1000))
        Y_s = A.dot(X_s) + np.asarray([ np.random.normal(0, np.eye(2))[:,0] for i in range(num_samps) ]).T
        Y_s_b = B.dot(X_s) + np.asarray([ np.random.normal(0, np.eye(2))[:,0] for i in range(num_samps) ]).T
        
        f_bin = []
        nf_bin = []
        
        for i in range(X_s.shape[1] - M):
            #for each sample
            nf_sum = [0.0]
            f_sum = [0.0]
            

            for k in range(M):
                #collect likelihood across lag
                nf = likelihood_point(np.expand_dims(X_s[:,i+k], axis=1), np.expand_dims(Y_s[:,i+k], axis=1), A, V_inv)
                f = likelihood_point(np.expand_dims(X_s[:,i+k], axis=1), np.expand_dims(Y_s_b[:,i+k], axis=1), A, V_inv)

                nf_sum.append(nf_sum[-1] + nf)
                f_sum.append(f_sum[-1] + f)


            if f_sum[-1] > 0:
                f_bin.append(1)
            else:
                f_bin.append(0)

            if nf_sum[-1] > 0:
                nf_bin.append(1)
            else:
                nf_bin.append(0)
                    
        pred = np.concatenate((f_bin, nf_bin))
        true = np.concatenate((np.ones(np.asarray(f_bin).shape), np.zeros(np.asarray(nf_bin).shape)))

        p = sklearn.metrics.precision_score(true, pred)
        r = sklearn.metrics.recall_score(true, pred)
                
        mc_output_by_lag_skew[M][j,1] = p
        mc_output_by_lag_skew[M][j,2] = r

In [ ]:
X_range = np.arange(0.0,3.5,0.1) 

M = lags[0]
clf = None #clear previous model
clf = KernelRidge(kernel="rbf", gamma = 1, degree=7, alpha=0.01)
f1 = 2*((mc_output_by_lag[M][:,1]*mc_output_by_lag[M][:,2])/(mc_output_by_lag[M][:,1] + mc_output_by_lag[M][:,2]))
plt.scatter(mc_output_by_lag[M][:,0], f1, marker="o", color=seaborn.xkcd_rgb["pale red"], s=10)
clf.fit(mc_output_by_lag[M][:,0].reshape(-1, 1), f1) 

r_curve = clf.predict(X_range.reshape(-1, 1))
r_curve[r_curve > 1] = 1

plt.plot(X_range, r_curve, color=seaborn.xkcd_rgb["pale red"], label="Identity covariance prior" + str(M), lw=2)


clf = None #clear previous model
clf = KernelRidge(kernel="rbf", gamma = 1, degree=7, alpha=0.01)
f1 = 2*((mc_output_by_lag_skew[M][:,1]*mc_output_by_lag_skew[M][:,2])/(mc_output_by_lag_skew[M][:,1] + mc_output_by_lag_skew[M][:,2]))
plt.scatter(mc_output_by_lag_skew[M][:,0], f1, marker="o", color=seaborn.xkcd_rgb["denim blue"], s=10)
clf.fit(mc_output_by_lag_skew[M][:,0].reshape(-1, 1), f1) 

r_curve = clf.predict(X_range.reshape(-1, 1))
r_curve[r_curve > 1] = 1

plt.plot(X_range, r_curve, color=seaborn.xkcd_rgb["denim blue"], label="Informed covariance prior" + str(M), lw=2)
    
#plt.title("F1 Score vs divergence of\n Monte Carlo realizations of B", fontsize=14)
plt.xlabel("$||A - B||_{F}$", fontsize=14)
plt.ylabel("F1 Score", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(loc=4)
plt.ylim(0,1.1)
plt.xlim(0.0,3.5)
plt.show()